In the implementation part of the module, you will in some cases be told exactly what to do, and in other cases you will have to come up with your own solution to a tricky problem. In the latter cases, it is crucial that you describe the choices you made and your reasoning behind them. 

# 2 - Implementation

## A) warm up

Let's assume that we pick a word completely randomly from the European parliament proceedings. According to your estimate, what is the probability that it is speaker? What is the probability that it is zebra?     

In [ ]:
from collections import Counter

# ten most common words for every language
not_english = ['europarl-v7.de-en.lc.de', 'europarl-v7.fr-en.lc.fr', 'europarl-v7.sv-en.lc.sv']

for language in not_english:
    text = ""
    with open(language) as stream:
        text = stream.read()
    words = text.split(" ") # string to list
    c= Counter(words) 
    print("For "+ language + " it's: ")
    print(c.most_common(10))

For europarl-v7.de-en.lc.de it's: 
[(',', 18549), ('die', 9649), ('der', 9139), ('und', 6920), ('in', 3934), ('zu', 3136), ('den', 2955), ('daß', 2725), ('von', 2448), ('für', 2432)]
For europarl-v7.fr-en.lc.fr it's: 
[('&apos;', 16729), (',', 15400), ('de', 14444), ('la', 9239), ('et', 6539), ('l', 6254), ('le', 5733), ('à', 5353), ('les', 5260), ('des', 5195)]
For europarl-v7.sv-en.lc.sv it's: 
[('att', 9138), (',', 8875), ('och', 6950), ('i', 5599), ('som', 4958), ('för', 4699), ('det', 4524), ('av', 3979), ('är', 3802), ('en', 3632)]


In [ ]:
#merging all english docs
tot_text = []
all_english = ['europarl-v7.de-en.lc.en', 'europarl-v7.fr-en.lc.en', 'europarl-v7.sv-en.lc.en']

for language in all_english:
    text=""
    with open(language) as stream:
        text = stream.read()
    words = text.split(" ")
    tot_text.extend(words)

c= Counter(tot_text)
print("For English it's: ")
print(c.most_common(10))

For English it's: 
[('the', 55362), (',', 42038), ('of', 28281), ('to', 26752), ('and', 21257), ('in', 17040), ('is', 13216), ('a', 12801), ('that', 12729), ('for', 8705)]


In [ ]:
the_parliament = []
the_parliament.extend(not_english)
the_parliament.extend(all_english)

tot_text =[]
for language in the_parliament:
    text=""
    with open(language) as stream:
        text = stream.read()
    words = text.split(" ")
    tot_text.extend(words)
    
c= Counter(tot_text)
print("For the whole parliament there is a: ")
print(str(c['speaker']/c.total()*100) + "% " + "prob for 'speaker'and a ")
print(str(c['zebra']/c.total()*100) + "% " + "prob for 'zebra'" )

For the whole parliament there is a: 
0.002003123658893535% prob for 'speaker'and a 
0.0% prob for 'zebra'


## B) Language Modeling  - bigram model  (loaded with the parliament)

Found a nice inspirational source: https://dev.to/amananandrai/language-model-implementation-bigram-model-22ij

In [ ]:
#Generate sentences from all data
the_parliament = []
the_parliament.extend(not_english)
the_parliament.extend(all_english)

complete_sentences =[]
for language in the_parliament:
    text=""
    with open(language) as stream:
        text = stream.read()
    words = text.split(" .\n")
    complete_sentences.extend(words)   

In [ ]:
#read in the data to compare
def readData(listOfSents):
    inData = listOfSents
    listOfWords=[]
    for i in range(len(inData)):
        for word in inData[i].split():
            if word != ",":         ## does this work as intended? 
                listOfWords.append(word)
                
    return listOfWords

In [ ]:
#creat bigram and unigram together with a list that hold them
def createBigram(data):
   listOfBigrams = []
   bigramCounts = {}
   unigramCounts = {}
   for i in range(len(data)-1):
      if i < len(data) - 1 and data[i+1].islower():

         listOfBigrams.append((data[i], data[i + 1]))

         if (data[i], data[i+1]) in bigramCounts:
            bigramCounts[(data[i], data[i + 1])] += 1  
         else:
            bigramCounts[(data[i], data[i + 1])] = 1

      if data[i] in unigramCounts:
         unigramCounts[data[i]] += 1
      else:
         unigramCounts[data[i]] = 1
   return listOfBigrams, unigramCounts, bigramCounts

In [ ]:
#compute the probability of words following each other
def calcBigramProb(listOfBigrams, unigramCounts, bigramCounts):
    listOfProb = {}
    for bigram in listOfBigrams:
        word1 = bigram[0]
        word2 = bigram[1]
        listOfProb[bigram] = (bigramCounts.get(bigram))/(unigramCounts.get(word1))
    return listOfProb

In [ ]:
#scripting some nice prints
if __name__ == '__main__':
    data = readData(complete_sentences)
    #print("\n HEY LOOK: ")
    #print(data[:20])
    listOfBigrams, unigramCounts, bigramCounts = createBigram(data)

    #print("\n Some possible Bigrams are ")
    #print(listOfBigrams[0:10])

    #print("\n A Bigram along with its frequency ")
    #print (str(listOfBigrams[0]) + ": ") 
    #print(bigramCounts.get(listOfBigrams[0]))

    #print("\n And the Unigram for \"" + str(listOfBigrams[0][0]) + "\" along with its frequency of: ")
    #print(unigramCounts.get(listOfBigrams[0][0]))

    bigramProb = calcBigramProb(listOfBigrams, unigramCounts, bigramCounts)

    #print("\n Bigrams along with their probability ")
    #print(bigramProb)
    inputList="ich erkläre die am freitag"
    splt=inputList.split()
    outputProb = 1
    bilist=[]
    bigrm=[]

    for i in range(len(splt) - 1):
        if i < len(splt) - 1:
            bilist.append((splt[i], splt[i + 1]))

    #print("\n The bigrams in given sentence are ")
    #print(bilist)
   
    for i in range(len(bilist)):
        if bilist[i] in bigramProb:
            outputProb *= bigramProb[bilist[i]]
        else:
            outputProb *= 0
    print('\n' + 'Probablility of sentence \"' + inputList + '\" = ' + str(outputProb))


Probablility of sentence "ich erkläre die am freitag" = 8.839638675847781e-09


What happens if you try to compute the probability of a sentence that contains a word that did not appear in the training texts? ANS: currently the probability is zero for that word and hence the sentence also has prob = 0. 

"Also if an unknown word comes in the sentence then the probability becomes 0. This problem of zero probability can be solved with a method known as Smoothing. In Smoothing, we assign some probability to unknown words also. Two very famous smoothing methods are: Laplace Smoothing & Good Turing" - Source of boilerplate code 

And what happens if your sentence is very long (e.g. 100 words or more)? ANS: many fairly small probabilities become minuscule, in the end practically zero. (fact check)

## C) Translation modeling

Self-check: if our goal is to translate from some language into English, why does our conditional probability seem to be written backwards? Why don't we estimate P(e|f) instead? 

ANS: Using Bayes rule we get that we can decompose the probability at hand to something that is proportional to it. The gain here is dividing tasks of keeping track of contents in the translation with P(F|E) and the task of getting grammar and fluency right with P(E). This also makes it easier to train the model since only English is required. I.e. this is division of labor, we get one language model and one translation model.  

TASK: Write code that implements the estimation algorithm for IBM model 1. Then print, for either Swedish, German, or French, the 10 words that the English word european is most likely to be translated into, according to your estimate. It can be interesting to look at this list of 10 words and see how it changes during the EM iterations.

### textParser class

In [ ]:
from os import linesep
import string
from collections import Counter, defaultdict
import numpy as np

dataset = {'de-en-de': 'europarl-v7.de-en.lc.de',
          'de-en-en': 'europarl-v7.de-en.lc.en',
          'fr-en-fr': 'europarl-v7.fr-en.lc.fr',
          'fr-en-en': 'europarl-v7.fr-en.lc.en',
          'sv-en-sv': 'europarl-v7.sv-en.lc.sv',
          'sv-en-en': 'europarl-v7.sv-en.lc.en',}

class text_parser:
    def __init__(self, data = dataset):
        self.data = data
        self.sentences = []
        self.words = {}
        self.keys = []

    def parse(self, keys):
        self.keys = keys
        for key in keys:
            with open(self.data[key], 'r') as file:
                s = file.read()
                self.sentences = s.split(" .\n")
                """
                sents = s.split(" .\n")
                for sent in sents:                # no need for looping through list and appending --> selfsentences = sents
                    self.sentences.append(sent)
                """ 
                words =  s.split(" ")
                self.words[key] = [word for word in words if word not in string.punctuation]
        
    def count(self, n = 10):
        for key in self.keys:
            most_common = Counter(self.words[key]).most_common(n)
            print('\nFor dataset: '+str(key) +', the most common words are:')
            print(*most_common, sep = "\n")
    
    def get_words(self, all_words = True, unique = False):
        if all_words: 
            all_words = [word for words in self.words.values() for word in words]
            if unique: return np.unique(all_words)
            else: return all_words
        else: return self.words
    
    def get_sent(self, n = -1):
        if n == -1: return self.sentences
        else: return self.sentences[:n]
  
    def prob_words(self,word_list):
        all_words = self.get_words(all_words = True)
        C = Counter(all_words)
        for word in word_list:
            if C[word] == 0: print(word+" is not used in: "+ str(self.keys))
            else: print("Probability of: "+ word +" in "+str(self.keys)+ " is: "+str(C[word]/C.total()))

   

In [ ]:
parser = text_parser()
parser.parse(['de-en-de', 'de-en-en'])
parser.count()


For dataset: de-en-de, the most common words are:
('die', 9649)
('der', 9139)
('und', 6920)
('in', 3934)
('zu', 3136)
('den', 2955)
('daß', 2725)
('von', 2448)
('für', 2432)
('ist', 2259)

For dataset: de-en-en, the most common words are:
('the', 18696)
('of', 9553)
('to', 9029)
('and', 7230)
('in', 5762)
('is', 4441)
('a', 4337)
('that', 4272)
('for', 2939)
('this', 2832)


In [ ]:
parser2 = text_parser()
parser2.parse(['fr-en-en', 'de-en-en','sv-en-en'])
parser2.prob_words(['speaker','zebra'])

Probability of: speaker in ['fr-en-en', 'de-en-en', 'sv-en-en'] is: 4.2331408750800126e-05
zebra is not used in: ['fr-en-en', 'de-en-en', 'sv-en-en']


In [ ]:
all_words = parser2.get_words(all_words =True)
print(all_words[:10])

all_words = parser2.get_words(all_words =False)
print(all_words)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### translationModel class

In [ ]:
import sys

MIN_PROB = sys.float_info.min
MIN_PROB


2.2250738585072014e-308

In [ ]:
class translationModel: 
    def __init__(self, orig_lang, trans_lang):
        
        #Initiate "orignal" & "translated" data
        orig_parser = text_parser()
        trans_parser = text_parser()

        #Parse "orignal" & "translated" data
        orig_parser.parse(orig_lang)
        trans_parser.parse(trans_lang)

        #Initiate sentences 
        self.orig_sent = orig_parser.get_sent()
        self.trans_sent = trans_parser.get_sent()
        """
        Are these below used at all? 
        #self.trans_probs = None
        self.orig_words = orig_parser.get_words(all_words=True, unique = True)
        self.trans_words = trans_parser.get_words(all_words=True, unique = True)
        """
    
        
    def calculate_translation_probs(self, n_iter=5, small_value = MIN_PROB, word=None):
        #initiate transition probabilities to some small value & word of interest
        self.trans_probs = defaultdict(lambda: defaultdict(lambda: small_value))
        word_of_interest = word
        if  (word_of_interest != None): 
            print("Using: " + word_of_interest + "\n")
        else: print("Error: Specify a word")
        """
        def get_similar_words(self, n = 10):
            word_probs = list(self.trans_probs[word_of_interest].items())
            word_probs.sort(key=lambda x: x[1], reverse=True)
            return word_probs[:10]
        """


        # print for each iteration 
        word_probs = list(self.trans_probs[word_of_interest].items())
        word_probs.sort(key=lambda x: x[1], reverse=True)
        print("\nRound: [" + str(0) + "/" + str(n_iter) + "]")
        print(*word_probs[:10], sep= "\n") 



        #Define # of EM iterations 
        for i in range(1,n_iter+1): # change to 0 - n_iter?   

            #Set all counts c(o,t) and c(t) to 0
            ot_count = defaultdict(lambda: defaultdict(lambda: small_value))
            o_count = defaultdict(lambda: small_value)

            #For each sentence pair
            for o_sent, t_sent in zip(self.orig_sent,self.trans_sent):

                o_words = o_sent.split()
                t_words = t_sent.split()

                #Include NULL word to original sentence
                o_words += ["NULL"]

                #For each original word (but here "t_word" due to bayes??)
                for t_word in t_words:

                    #Get (initilize?) transition probability sum
                    tp_sum = 0

                    for o_word in o_words:         # load sum
                        tp_sum+=self.trans_probs[o_word][t_word]

                    #For each translated word (and null word)
                    for o_word in o_words:

                        #Compute alignment probability
                        align_prob = self.trans_probs[o_word][t_word]/tp_sum

                        #Update Pseudocount of c(o,t)& of c(t)
                        ot_count[o_word][t_word] += align_prob
                        o_count[o_word] += align_prob

            # Reestimate transition probabilities
            for o_word, ot_dict in ot_count.items():
                for t_word, _ in ot_dict.items():
                    self.trans_probs[o_word][t_word] = ot_count[o_word][t_word] / o_count[o_word]
            
            # print for each iteration 
            word_probs = list(self.trans_probs[word_of_interest].items())
            word_probs.sort(key=lambda x: x[1], reverse=True)
            print("\nRound: [" + str(i) + "/" + str(n_iter) + "]")
            print(*word_probs[:10], sep= "\n") 
            


In [ ]:
model = translationModel(['sv-en-en'],['sv-en-sv'])
model.calculate_translation_probs(word="european")


Using: european


Round: [0/5]


Round: [1/5]
('att', 0.03681418155342775)
(',', 0.03634172163664544)
('och', 0.026906798671119368)
('i', 0.0247574372369237)
('det', 0.022179298361308157)
('som', 0.020877887365033796)
('för', 0.01892124782529525)
('av', 0.018094315236971116)
('en', 0.017186765112304034)
('är', 0.015331287767306873)

Round: [2/5]
(',', 0.06324136464695101)
('att', 0.06284287431842107)
('och', 0.04364733168529026)
('i', 0.04261401110938812)
('det', 0.03628207117599448)
('som', 0.03585449599160646)
('av', 0.032433891994174055)
('en', 0.03113832865372013)
('för', 0.03022247936020297)
('vi', 0.025753582049213894)

Round: [3/5]
(',', 0.07156659907184448)
('att', 0.06940639207958753)
('i', 0.04869083204289162)
('och', 0.04637430821202799)
('som', 0.04107332752682835)
('av', 0.0397542748487959)
('det', 0.03951818948130992)
('en', 0.0387084939374777)
('för', 0.032329109509809474)
('vi', 0.031845587458155515)

Round: [4/5]
(',', 0.07456801556309861)
('att', 0.07066257106302784)


Fungerar uppebarligen inte. Kolla: så att vår taktik med uniform standard slh fungerar. Kolla slg för ord som "europeisk". Jämför beräkningarna och looparna med Git. 

Possible compare to: https://github.com/lukaborec/IBM-Model-1/blob/master/IBM1.py

## D) Decoding

Because it requires sifting through a huge search space of potential phrase outputs, the task of locating the sentence with the highest probability in a machine translation decoder model is seen as being algorithmically challenging. It is a combinatorially heavy problem. It can be challenging to discover the ideal solution in a fair amount of time because of the complexity of language and the wide variety of sentence structures and phrasing.

Using a probabilistic model to estimate the likelihood of a sentence given the source sentence, can result in many possible outputs with high probabilities. Hence, finding the sentence with the highest probability is not necessarily a guarantee that is the most accurate and coherent translation. 

Hence, let's introduce: 

- To complete a translation in a timely manner, we only consider the top 20 translations for a word/phrase. I.e. we will satisfy with getting one of the sentences with the highest probability, not necessarily the one and only with the highest probability.  

- Markov property: the following word only depends on the previous one, allowing us to use a bigram model to gather more contextual information by computing probabilities from adjacent words. This introduces an assumption of independence. 

- Caveat: using this model with a certain corpus of language and words limits the translation to words already existing in it.  

In [ ]:
"""Given that we pass a word into the get_translation_probs 
--> do this for every word in the source sentence

Implement a connection to adjacent word, remembering the 1-2 last words for better "contextual understanding"


"""







'Given that we pass a word into the get_translation_probs --> do this for every word in the source sentence\n\n\n\n'

## 

# 3 - Discussion 

## a) Different evaluation protocols for machine translation systems

BLEU (Bilingual Evaluation Understudy): A popular metric that examines the n-gram overlap between the predicted sentence and one or more reference translations is called BLEU (Bilingual Evaluation Understudy). The benefit of BLEU is that it can be used to compare various machine translation algorithms and is straightforward and effective to calculate. However, it has the disadvantage of being overly influenced by the length of the predicted sentence and the presence of specific n-grams, and can sometimes give high scores to translations that are semantically incorrect.

METEOR (Metric for Evaluation of Translation with Explicit Ordering): This metric assesses semantic similarity more thoroughly than BLEU by taking word-level alignments and synonymy into account. METEOR's benefit is that it offers a more complex assessment of translation quality. However, calculating it costs more computationally than with BLEU.

Humans in the loop is a third alternative. It basically means that humans are used to evaluating the quality of the translation. The perhaps biggest advantage is the direct measure of quality from the end-user. Although carrying the potential for some really high-quality assessments, in line with the capability of the human at issue, it has some challenges of its own. For example, it is expensive in terms of resources, and not very replicable, further it questionable reliability as the individual and their preferences and biases can easily affect it. This method is the most subjective in its evaluation. 

We still have: ROUGE (Recall-Oriented Understudy for Gisting Evaluation), TER (Translation Error Rate):

## b) Translating from Estonian

The technical reason why the translation results in a male or female pronoun is that it is related to an overrepresentation of one gender in the respective professional role. E.g. the role of a doctor is probably more often associated with male pronouns such as "he/him" in the data that the model builds on, and similar is the role of a nurse more often combined with "she/her" in those examples. The underlying reason for this vocabulary is the real-world overrepresentation (current or historical) that one gender is more commonly associated with a certain profession.  We believe that this is a natural bias that exists in the language and it is a historical remnant, despite being based on sometimes rather outdated assumptions of gender roles, the translation itself is still a guess based on the highest probability for how the pronoun is expected to be found. At least according to the data set used. 

Hence, if the data set is big and broad enough to represent the modern language usage of the most recent years, then the qualified guess and decision to choose one pronoun over the other based on the likelihood of it in combination with a certain role should be seen as a feature rather than a bug. Adjusting this by letting the word "Ta" be translated to he's/she's with a 50/50 chance instead would most likely not result in any better translations than in the current situation.     

## c) Bat VS Bat

The reason why the first two translations are correct is probably that they are more distinct and unambiguous and hence more in line with one or the other meaning of the homograph "bat" the examples found in the word space(farlig användning av space?) that the model builds on. I.e. there are more examples of the word bat being translated into "slagträ" in the vicinity of the words like "hits" and "ball", giving it a higher probability to be correct than "fladdermus" in these cases. Similar is the case for the second sentence, where "bat" is more likely to be aiming at the animal bat in the context of eating insects. 

In the third sentence, however, the situation is a bit different. Here the context and adjacent word would suggest that the bat again is an animal since it "lives" somewhere. Nevertheless, the model seems not to be able to distinguish this as it has no sense of the actual meaning of words like "living". Instead, it completely relies on having seen examples of similar use cases of the word "bat" in a sentence and knowing what the word was translated into in that specific setting. The reason why it is a blend of the two cases of the animal and the baseball bat could be explained with the assumption that the bat should refer to the same thing, the same version of "bat", as in the previous sentences in one and the same text. But this is more of a speculation from our side than it is a fact, since we don't know exactly how the translation model of google is built. For us, it doesn't make sense to translate the homonym into some morphed version of its different meanings, but the model seems to have done so as a result of not knowing which version was more probable.    

# 3 - Discussion - new version

## a) Different evaluation protocols for machine translation systems

BLEU (Bilingual Evaluation Understudy): A popular metric that examines the n-gram overlap between the predicted sentence and one or more reference translations is called BLEU (Bilingual Evaluation Understudy). The benefit of BLEU is that it can be used to compare various machine translation algorithms and is straightforward and effective to calculate. However, it has the disadvantage of being overly influenced by the length of the predicted sentence and the presence of specific n-grams, and can sometimes give high scores to translations that are semantically incorrect.

METEOR (Metric for Evaluation of Translation with Explicit Ordering): This metric assesses semantic similarity more thoroughly than BLEU by taking word-level alignments and synonymy into account. METEOR's benefit is that it offers a more complex assessment of translation quality. However, calculating it costs more computationally than with BLEU.

Humans in the loop is a third alternative. It basically means that humans are used to evaluating the quality of the translation. The perhaps biggest advantage is the direct measure of quality from the end-user. Although carrying the potential for some really high-quality assessments, in line with the capability of the human at issue, it has some challenges of its own. For example, it is expensive in terms of resources, and not very replicable, further it questionable reliability as the individual and their preferences and biases can easily affect it. This method is the most subjective in its evaluation. 

We still have: ROUGE (Recall-Oriented Understudy for Gisting Evaluation), TER (Translation Error Rate):

## b) Translating from Estonian

The technical reason why the translation results in a male or female pronoun is that it is related to an overrepresentation of one gender in the respective professional role. E.g. the role of a doctor is probably more often associated with male pronouns such as "he/him" in the data that the model builds on, and similar is the role of a nurse more often combined with "she/her" in those examples. The underlying reason for this vocabulary is the real-world overrepresentation (current or historical) that one gender is more commonly associated with a certain profession.  We believe that this is a natural bias that exists in the language and it is a historical remnant, despite being based on sometimes rather outdated assumptions of gender roles, the translation itself is still a guess based on the highest probability for how the pronoun is expected to be found. At least according to the data set used. 

If the data set is big and broad enough to represent the language as a whole and if it reflects the skewed reality that the language is part of, then the decision to predict one pronoun over the other could be seen as a more accurate and helpful way of designing the system. However, there is no way of telling what version of the pronoun should be in any particular translation unless this is somehow captured from the rest of the text. Assuming one gender over the other based on the roles alone is rather discriminating and does not reflect the progressive Europe and western world that Estonia is part of. That way it can also be viewed as a bug. 

## c) Bat VS Bat

The reason why the first two translations are correct is probably that they are more distinct and unambiguous and hence more in line with one version of the homograph "bat" that exists in the "word space" of the model. I.e. there are more examples of the word bat being translated into "slagträ" in the vicinity of the words like "hits" and "ball", giving it a higher probability to be correct than "fladdermus" in these cases. Similar is the case for the second sentence, where "bat" is more likely to be aiming at the animal bat in the context of eating insects. In the third sentence, however, the situation is a bit different. Here the context and adjacent word would suggest that the bat again is an animal since it "lives" somewhere. Nevertheless, the model seems not to be able to distinguish this as it has no sense of the actual meaning of words like "living". 

(Instead, it completely relies on having seen examples of similar use cases of the word "bat" in a sentence and knowing what the word was translated into in that specific setting.) The reason why it is a blend of the two cases of the animal and the baseball bat could be explained by the model assuming that the bat should refer to the same thing, the same version of "bat", as in the previous sentences in one and the same text/paragraph. Or google could have some sort of loss function that tries to interpolate a new third version of the previously seen two versions to optimize the translation. The model could use some error measures like edit distance and have concluded that this new string had the least error. Anyway, different from our model which only can translate into words from its vocabulary, this system seems to be able to create new words. Maybe as part of some adaptive learning.  But all this is speculation from our side, since we don't know exactly how the translation model of google is built. For us, it doesn't make sense to translate the homonym into some morphed version of its different meanings.    

Contrast to our model --> could only us words in the data. 

Google's model might use some sort of deep learning where the maximized score for the model is this morphed version. 

Error measurement: comparing strings with "edit distance". 

Unsupervised learningwe wwwwww. Pros (adaptive for new language, possible to catch up), cons(creating nonsensical words) --> reviews good in the sense to human, 

Kan vara interpolation: --> e.g mnist, kombinera två handskrivna variabler --> skicka in i encoder --> decoder skickar ut "syntetisk" siffra

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=27527141-1c2d-41eb-a7d6-3699f108d4e9' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>